## Pacejka Formula


The aim of this project is to use the magic formula to fit curves to tire performance based on the emperical data given. We'll use these curves to determine the best tire to use during competition. 

In [1]:
import math
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt

### Magic Longitudinal Formula
This is a mathematical model used to calculate the longitudinal force of a tire based on its vertical load and longitudinal slip.

In [2]:
# SR = slip ratio
# B = stiffness factor
# C = shape factor
# D = peak factor
# E = curvature factor
def longitudinal_formula(SR, B, C, D, E):
    D * np.sin(C * np.arctan(B * (1 - E) * SR + E * np.arctan(B * SR)))

### Magic Lateral Force Formula
This model describes the relationship between the lateral force and slip angle.

In [3]:
# SA = slip angle
# B = stiffness factor
# C = shape factor
# D = peak factor
# E = curvature factor
def lateral_formula(SA, B, C, D, E):
    SA_rad = np.radians(SA)
    return D * np.sin(C * np.arctan(B * (1 - E) * SA_rad + E * np.arctan(B * SA_rad)))

### Data Processing
We will now process the data from the .dat files provided by FSAE.

In [9]:
# Load in the data
csv_path = r"/Users/EmmyChen/IdeaProjects/pacejka-tire-modeling/research/B1965raw14.dat"
data = pd.read_csv(csv_path, sep='\t')

/var/folders/7l/nck1kh_j2rv3w8xvm790mj440000gn/T/ipykernel_58160/1869420761.py:3: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(csv_path, sep='\t')


In [10]:
data.head()

,,,,,,,,,,,,,,,,,,,,TIRF Data File: Project 1965; Run 014; Date 07-31-2018; Time 07:37; ISO False; ID 7jsWp80
ET,V,N,SA,IA,RL,RE,P,FX,FY,FZ,MX,MZ,NFX,NFY,RST,TSTI,TSTC,TSTO,AmbTmp,SR
s,kph,rpm,deg,deg,cm,cm,kPa,N,N,N,Nm,Nm,none,none,deg c,deg c,deg c,deg c,deg c,SAE
0.000,0.05,0.08,0.007,0.010,19.62,154.09,83.72,5.22,-59.60,-605.62,-4.02,-0.61,-0.0086,0.0984,29.79,26.07,26.67,27.61,26.73,-0.8727
0.010,0.07,0.01,0.013,-0.006,19.63,1285.57,83.83,5.66,-60.77,-602.97,-5.38,-0.58,-0.0094,0.1008,29.78,26.07,26.65,27.62,26.71,-0.9847
0.020,0.06,0.15,0.007,-0.007,19.63,100.46,83.42,5.64,-60.76,-601.08,-6.26,-0.59,-0.0094,0.1011,29.77,26.06,26.65,27.60,26.73,-0.8046


In [11]:
# Extract relevant columns
SR = data['SR'].values
FX = data['FX'].values
SA = data['SA'].values
FY = data['FY'].values
FZ = data['FZ'].values

KeyError: 'SR'